<div class="alert alert-block alert-info">
<b>Tip:</b> Install tbe jupyter notebook first.
</div>

# Installing Jupyter Notebook in terminal

```bash
# Navigate to the home directory
cd ~

# Install Jupyter Notebook using pip3
pip3 install jupyter


# Plot for those columns with less than 1006 missing value

In [13]:
# import modulus
import pandas as pd
# import os
# script_path = 'missing_table.py'
# os.system(f'python3 {script_path}')
# !python missing_table.py
from missing_table import missing_value_df
from datetime import datetime
import matplotlib.pyplot as plt

In [11]:
# This function converts the string into the type of datetime %d/%m/%Y
def convert_date_format(date_str):
    try:
        date_time_obj = datetime.strptime(date_str, '%d/%m/%Y %H:%M')
        formatted_date = date_time_obj.strftime('%m/%d/%Y')
        return formatted_date
    except ValueError:
        pass

    try:
        date_time_obj = datetime.strptime(date_str, '%d/%m/%Y')
        return date_str
    except ValueError:
        raise ValueError(f"Unsupported date format: {date_str}")

In [16]:
# note: More needs to be added to this script to clean the columns as needs be
# edit: 

# edit: changed filed path
df = pd.read_csv("../raw_data/australia.csv",
                 na_values=[" ", "__NA__"], keep_default_na = True, low_memory=False)
# Set the threshold for missing value count
# edit: change cound to 1006
thresh_value = 1006

# Extract the variable names with missing value counts larger than 1006
columns_to_drop = missing_value_df.loc[missing_value_df['Missing Value Count']
                                       > thresh_value, 'Variable Name'].tolist()

df.drop(columns=columns_to_drop, inplace=True)
df.dropna(inplace=True)

# create a new column in the csv that computer from week 1 for every two weeks 
df["endtime"] = df["endtime"].apply(convert_date_format)
df["endtime"] = pd.to_datetime(df["endtime"])

# Find the start date (minimum date) and end date (maximum date)
start_date = df['endtime'].min()
end_date = df['endtime'].max()

# Create a new column 'week_number' and assign week numbers
df['week_number'] = ((df['endtime'] - start_date).dt.days // 14) + 1

print(df)

       RecordNo    endtime    qweek i9_health        i11_health i12_health_1  \
0             0 2020-04-01   week 1       Yes      Very willing       Rarely   
1             1 2020-04-01   week 1       Yes      Very willing   Not at all   
2             2 2020-04-01   week 1       Yes      Very willing       Always   
3             3 2020-04-01   week 1       Yes      Very willing       Always   
4             4 2020-04-01   week 1       Yes  Somewhat willing   Not at all   
...         ...        ...      ...       ...               ...          ...   
52822     52822 2022-03-16  week 53        No  Somewhat willing   Frequently   
52823     52823 2022-03-16  week 53  Not sure      Very willing    Sometimes   
52824     52824 2022-03-16  week 53  Not sure      Very willing   Not at all   
52825     52825 2022-03-16  week 53       Yes      Very willing   Frequently   
52826     52826 2022-03-16  week 53        No      Very willing       Always   

      i12_health_2 i12_health_3 i12_hea

In [17]:
# note: More needs to be added to this script to clean the columns as needs be

# edit: changed filed path
df = pd.read_csv("../raw_data/australia.csv",
                 na_values=[" ", "__NA__"], keep_default_na = True, low_memory=False)
# Set the threshold for missing value count
# edit: change cound to 10029
thresh_value = 10029

# Extract the variable names with missing value counts larger than 10029
columns_to_drop = missing_value_df.loc[missing_value_df['Missing Value Count']
                                       > thresh_value, 'Variable Name'].tolist()

df.drop(columns=columns_to_drop, inplace=True)
df.dropna(inplace=True)

# create a new column in the csv that computer from week 1 for every two weeks 
df["endtime"] = df["endtime"].apply(convert_date_format)
df["endtime"] = pd.to_datetime(df["endtime"])

# Find the start date (minimum date) and end date (maximum date)
start_date = df['endtime'].min()
end_date = df['endtime'].max()

# Create a new column 'week_number' and assign week numbers
df['week_number'] = ((df['endtime'] - start_date).dt.days // 14) + 1

print(df)

       RecordNo    endtime    qweek  i2_health i9_health  \
9023       9023 2020-06-24  week 10        0.0  Not sure   
9024       9024 2020-06-24  week 10        2.0        No   
9025       9025 2020-06-24  week 10        6.0       Yes   
9026       9026 2020-06-24  week 10       20.0       Yes   
9027       9027 2020-06-24  week 10        0.0       Yes   
...         ...        ...      ...        ...       ...   
51826     51826 2022-02-26  week 52        6.0        No   
51827     51827 2022-02-26  week 52        4.0       Yes   
51828     51828 2022-03-01  week 52        3.0       Yes   
51829     51829 2022-03-01  week 52       10.0       Yes   
51830     51830 2022-03-01  week 52      100.0        No   

                          i11_health i12_health_1 i12_health_2 i12_health_3  \
9023                        Not sure    Sometimes       Rarely    Sometimes   
9024                    Very willing   Frequently    Sometimes    Sometimes   
9025                    Very willing   Not

In [ ]:
# Save the cleaned DataFrame to a new CSV file
# edit: fixed file path/save location
df.to_csv("../data/cleaned_data.csv", index=False)